In [5]:
import pandas as pd
import numpy as np
import json
from datasets import load_dataset
substan_train  = []
substan_test = []

with open('../data/substan-review/train.jsonl', 'r') as f:
    for line in f:
        substan_train.append(json.loads(line))

with  open('../data/substan-review/test.jsonl', 'r') as f:
    for line in f:
        substan_test.append(json.loads(line))




In [6]:
reviewer2_data = load_dataset("GitBag/Reviewer2_PGE_cleaned")

Generating train split: 76273 examples [00:07, 10149.36 examples/s]
Generating validation split: 9621 examples [00:01, 7328.75 examples/s]
Generating test split: 10594 examples [00:01, 8080.64 examples/s]


In [16]:
substan_reviews = []

for i in range(len(substan_train)):
    substan_reviews.append(substan_train[i]['review'])
for i in range(len(substan_test)):
    substan_reviews.append(substan_test[i]['review'])

print(len(substan_reviews))

550


In [33]:
for x in reviewer2_data:
    print(x)

train
validation
test


In [49]:
from tqdm import tqdm
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from rapidfuzz import fuzz
matches = 0
normalized_levenshtein = NormalizedLevenshtein()
match_dict = {}
not_found_ids = []
for i in tqdm(range(len(substan_reviews))):
    # for j in tqdm(range(len(reviewer2_data['train']['review']))):
    found = False
    for split in reviewer2_data:

        for review in reviewer2_data[split]['review']:
            s1 = substan_reviews[i].lower()
            s2 = review.lower()
            if abs(len(s1) - len(s2)) > 100:
                continue
            # distance = normalized_levenshtein.distance(s1, s2)
            sim = fuzz.ratio(s1, s2)
            if sim >= 70.0:
                match_dict[i] = split
                matches += 1
                found = True
                break
        if found:
            break
    
    if not found:
        not_found_ids.append(i)
        


100%|██████████| 550/550 [13:16<00:00,  1.45s/it]


In [50]:
matches

536

In [57]:
asap_data = []

with open('../data/asap/dataset/aspect_data/review_with_aspect.jsonl', 'r') as f:
    for line in f:
        asap_data.append(json.loads(line))

aspects = {}

for i in range(len(asap_data)):
    t = asap_data[i]['text']
    for label in asap_data[i]['labels']:
        aspect = label[2]
        begin = label[0]
        end = label[1]
        if aspect not in aspects:
            aspects[aspect] = []
        
        aspects[aspect].append(t[begin:end+1])

In [68]:
from random import sample 

rep_neg = sample(aspects['replicability_negative'], 10)
rep_pos = sample(aspects['replicability_positive'], 10)
comp_neg = sample(aspects['meaningful_comparison_negative'], 10)
comp_pos = sample(aspects['meaningful_comparison_positive'], 10)


In [69]:
rep_neg = [[x, 'replicability_negative'] for x in rep_neg]
rep_pos = [[x, 'replicability_positive'] for x in rep_pos]
comp_neg = [[x, 'meaningful_comparison_negative'] for x in comp_neg]
comp_pos = [[x, 'meaningful_comparison_positive'] for x in comp_pos]

df = pd.DataFrame(rep_neg + rep_pos + comp_neg + comp_pos, columns=['text', 'aspect'])
df.to_csv('asap_sample.csv', index=False)

NameError: name 'load_dataset' is not defined